In [11]:
import requests
import time

NODES = {
    'A': 'http://localhost:3000',
    'B': 'http://localhost:3001',
    'C': 'http://localhost:3002'
}

def get_balance(node):
    return requests.get(f'{node}/api/wallet-info').json()

def send_transaction(sender_node, recipient_address, amount):
    response = requests.post(f'{sender_node}/api/transact', json={
        'recipient': recipient_address,
        'amount': amount
    })
    return response.json()

def get_transaction_pool(node):
    return requests.get(f'{node}/api/transaction-pool-map').json()

def mine_block(node):
    return requests.get(f'{node}/api/mine-transactions').json()

def get_latest_block(node):
    return requests.get(f'{node}/api/blocks').json()[-1]

def test_cross_node_transaction():
    print("🔐 Fetching Node A and B wallet addresses...")
    wallet_a = get_balance(NODES['A'])['address']
    
    # Hardcoded public key of Node B (must be from /api/wallet-info)
    wallet_b = "040fa6c4af772ea82d24164271be605e016795f9eab0791d6230e3a495a6946d929c65966dcbf49b9684608fdcfd7519c1a71c991a4ace50c26843b7205d1a9512"

    print("💰 Balances BEFORE transaction:")
    balances_before = {
        'A': get_balance(NODES['A'])['balance'],
        'B': get_balance(NODES['B'])['balance'],
        'C': get_balance(NODES['C'])['balance']
    }
    print(balances_before)

    print("💸 Node A sending 20 coins to Node B...")
    tx_response = send_transaction(NODES['A'], wallet_b, 20)
    if tx_response.get("type") != "success":
        print("❌ Transaction failed:", tx_response.get("message"))
        return
    print("✅ Transaction broadcasted.")

    print("⏳ Waiting for pub/sub sync...")
    time.sleep(2)

    print("🔎 Verifying transaction pool on all nodes...")
    for label, url in NODES.items():
        pool = get_transaction_pool(url)
        if any(tx['input']['address'] == wallet_a for tx in pool.values()):
            print(f"✅ Node {label}: Transaction found in pool.")
        else:
            print(f"❌ Node {label}: Transaction MISSING from pool.")

    print("⛏️ Node C mining the transaction...")
    result = mine_block(NODES['C'])
    if not result.get("success"):
        print("❌ Mining failed:", result.get("reason"))
        return
    print("✅ Mining successful.")

    print("⏳ Waiting for chain propagation...")
    time.sleep(2)

    print("📦 Verifying all nodes have the same latest block...")
    latest_blocks = {label: get_latest_block(url) for label, url in NODES.items()}
    hashes = [block['hash'] for block in latest_blocks.values()]
    if len(set(hashes)) == 1:
        print("✅ All nodes have the same latest block.")
    else:
        print("❌ Chain mismatch!")
        for label, block in latest_blocks.items():
            print(f"Node {label}: {block['hash']}")

    print("💰 Balances AFTER transaction:")
    balances_after = {
        'A': get_balance(NODES['A'])['balance'],
        'B': get_balance(NODES['B'])['balance'],
        'C': get_balance(NODES['C'])['balance']
    }
    print(balances_after)

    # Mining reward check
    mining_reward = balances_after['C'] - balances_before['C']
    if mining_reward >= 49.99:
        print(f"✅ Node C received the mining reward: +{mining_reward:.2f} coins.")
    else:
        print(f"❌ Mining reward missing or incorrect for Node C. Only +{mining_reward:.2f} coins.")

    # Validate balance change for A and B
    if balances_after['A'] < balances_before['A'] and balances_after['B'] > balances_before['B']:
        print("✅ Balances updated correctly.")
    else:
        print("❌ Balance inconsistency detected.")

if __name__ == "__main__":
    test_cross_node_transaction()


🔐 Fetching Node A and B wallet addresses...
💰 Balances BEFORE transaction:
{'A': 1000, 'B': 1020, 'C': 1120}
💸 Node A sending 20 coins to Node B...
✅ Transaction broadcasted.
⏳ Waiting for pub/sub sync...
🔎 Verifying transaction pool on all nodes...
✅ Node A: Transaction found in pool.
✅ Node B: Transaction found in pool.
✅ Node C: Transaction found in pool.
⛏️ Node C mining the transaction...
✅ Mining successful.
⏳ Waiting for chain propagation...
📦 Verifying all nodes have the same latest block...
✅ All nodes have the same latest block.
💰 Balances AFTER transaction:
{'A': 980, 'B': 1040, 'C': 1170}
✅ Node C received the mining reward: +50.00 coins.
✅ Balances updated correctly.
